In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import types
import copy

Using TensorFlow backend.


In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [5]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [6]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [7]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [8]:
db = DFDB('../trial/gradientboosting.pkl', auto_commit=False)

In [10]:
df_trial = db.select()
df_trial['kfold'] = df_trial['param'].apply(lambda x: x['kfold'])

In [12]:
df_trial[['datetime','nfeatures', 'kfold', 'remark', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff','remark']].loc[0:19]

,datetime,nfeatures,kfold,remark,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff,remark
0,2019-05-10 10:08:45.942462,1071,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.818027,0.010086,2.195905,0.527265,0.377878,None
1,2019-05-10 12:36:44.746797,200,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.819021,0.009948,2.184827,0.523375,0.365805,None
2,2019-05-10 12:44:53.047447,190,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.820537,0.010106,2.179309,0.525266,0.358772,None
3,2019-05-10 12:52:49.016791,180,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.821202,0.010218,2.174044,0.530444,0.352842,None
4,2019-05-10 13:00:23.427527,170,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.822041,0.009875,2.171204,0.531807,0.349163,None
5,2019-05-10 13:07:32.021287,160,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.826642,0.010263,2.168563,0.541657,0.341921,None
6,2019-05-10 13:14:10.965752,150,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.830268,0.010180,2.162702,0.531846,0.332434,None
7,2019-05-10 13:20:35.009869,140,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.834635,0.010421,2.145930,0.539925,0.311295,None
8,2019-05-10 13:26:21.571473,130,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.835565,0.010528,2.147050,0.537258,0.311484,None
9,2019-05-10 13:31:45.137073,120,"{'n_splits': 8, 'random_state': 1985, 'shuffle': True, 'type': 'group'}",None,1.838652,0.011059,2.141269,0.548358,0.302617,None


In [13]:
param_idx = 19
column_idx = 1
db_ = db
df_trial_ = df_trial
mytrial = []
columns = copy.deepcopy(df_trial_.loc[column_idx]['param']['columns'])
param = copy.deepcopy(df_trial_.loc[param_idx]['param'])
# param['algorithm']['init'] = {'logging_level': 'Silent', **param['algorithm']['init']}
param['scaler']['init'] = {}
param['algorithm']['fit'] = {}

selected_columns = EP.revert_rfe(df_train, param, columns, df_test, mytrial, start_columns=[columns[0]], limit=20, remark='start from top1 column')
print(len(selected_columns))
for trial_i in mytrial:
    db_.insert(trial_i)
df_trial_ = db_.select()
df_trial_['kfold'] = df_trial_['param'].apply(lambda x: x['kfold'])

20


In [15]:
df_trial_[df_trial_['remark']=='start from top1 column'][['datetime','remark','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
327,2019-05-22 10:40:31.223506,start from top1 column,1,2.057523,4.946885e-06,2.070607,0.000050,0.013083
328,2019-05-22 10:40:33.360013,start from top1 column,2,2.004518,1.821883e-05,2.021601,0.000075,0.017082
329,2019-05-22 10:40:36.041150,start from top1 column,3,1.997760,1.288089e-05,2.017327,0.000085,0.019567
330,2019-05-22 10:40:39.225572,start from top1 column,4,1.993438,1.092364e-05,2.014509,0.000089,0.021071
331,2019-05-22 10:40:42.982222,start from top1 column,5,1.991495,6.889197e-06,2.013376,0.000082,0.021882
332,2019-05-22 10:40:47.272480,start from top1 column,6,1.989967,9.345537e-06,2.012571,0.000086,0.022604
333,2019-05-22 10:40:52.115516,start from top1 column,7,1.981613,1.492596e-05,2.005958,0.000099,0.024345
334,2019-05-22 10:40:57.477657,start from top1 column,8,1.976686,7.288181e-06,2.001152,0.000083,0.024467
335,2019-05-22 10:41:03.185964,start from top1 column,9,1.976784,9.264981e-06,2.002551,0.000115,0.025767
336,2019-05-22 10:41:09.295947,start from top1 column,9,1.972328,3.312993e-06,1.999440,0.000064,0.027111


In [14]:
db_.commit()

In [ ]:
mytrial =[]
columns_ = selected_columns

###  tune hypterparameters
def objective(trial):
    
    max_depth = trial.suggest_int('max_depth', 2, 6)
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.4)
    n_estimators = trial.suggest_int('n_estimators', 100,500)
    subsample = trial.suggest_uniform('subsample', 0.6, 1.0)
    alpha = trial.suggest_uniform('alpha', 0.00001, 1.0)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':columns_,
        'kfold':{
            'n_splits': 8,
            'random_state': 1985,
            'shuffle': True,
            'type': 'group'
        },
        'scaler':{
            'cls':'StandardScaler',
            'init':{}
        },
        'algorithm':{
            'cls':'GradientBoostingRegressor',
            'init':{
                "max_depth":max_depth,
                "learning_rate":learning_rate,
                "n_estimators":n_estimators,
                "subsample":subsample,
                "alpha":alpha,
                "random_state":random_state,
            },
            'fit':{
            },
        },
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 545')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

[I 2019-05-24 02:08:18,049] Finished a trial resulted in value: 1.6653006923565812. Current best value is 1.6653006923565812 with parameters: {'max_depth': 6, 'learning_rate': 0.1907970620683256, 'n_estimators': 134, 'subsample': 0.8358287669775222, 'alpha': 0.033813787330786535, 'random_state': 1807}.
[I 2019-05-24 02:17:05,327] Finished a trial resulted in value: 3.138505409871065. Current best value is 1.6653006923565812 with parameters: {'max_depth': 6, 'learning_rate': 0.1907970620683256, 'n_estimators': 134, 'subsample': 0.8358287669775222, 'alpha': 0.033813787330786535, 'random_state': 1807}.
[I 2019-05-24 02:18:40,905] Finished a trial resulted in value: 0.5093016528043659. Current best value is 0.5093016528043659 with parameters: {'max_depth': 3, 'learning_rate': 0.17700944205567065, 'n_estimators': 129, 'subsample': 0.9565982042793041, 'alpha': 0.9949241613549263, 'random_state': 7189}.
[I 2019-05-24 02:20:28,674] Finished a trial resulted in value: 0.39415964984193647. Curre

[I 2019-05-24 03:40:29,169] Finished a trial resulted in value: 0.6534183687174804. Current best value is 0.12909224996449542 with parameters: {'max_depth': 3, 'learning_rate': 0.013376621399495775, 'n_estimators': 105, 'subsample': 0.6204899998086485, 'alpha': 0.2699165905935789, 'random_state': 9758}.
[I 2019-05-24 03:42:30,451] Finished a trial resulted in value: 0.784256050757786. Current best value is 0.12909224996449542 with parameters: {'max_depth': 3, 'learning_rate': 0.013376621399495775, 'n_estimators': 105, 'subsample': 0.6204899998086485, 'alpha': 0.2699165905935789, 'random_state': 9758}.
[I 2019-05-24 03:45:07,268] Finished a trial resulted in value: 1.4170402033677176. Current best value is 0.12909224996449542 with parameters: {'max_depth': 3, 'learning_rate': 0.013376621399495775, 'n_estimators': 105, 'subsample': 0.6204899998086485, 'alpha': 0.2699165905935789, 'random_state': 9758}.
[I 2019-05-24 03:47:45,282] Finished a trial resulted in value: 1.016546802260916. Cur

[I 2019-05-24 04:44:14,277] Finished a trial resulted in value: 0.3190617914726099. Current best value is 0.12909224996449542 with parameters: {'max_depth': 3, 'learning_rate': 0.013376621399495775, 'n_estimators': 105, 'subsample': 0.6204899998086485, 'alpha': 0.2699165905935789, 'random_state': 9758}.
[I 2019-05-24 04:46:30,427] Finished a trial resulted in value: 0.7514380891421499. Current best value is 0.12909224996449542 with parameters: {'max_depth': 3, 'learning_rate': 0.013376621399495775, 'n_estimators': 105, 'subsample': 0.6204899998086485, 'alpha': 0.2699165905935789, 'random_state': 9758}.
[I 2019-05-24 04:48:29,099] Finished a trial resulted in value: 0.3037876499342833. Current best value is 0.12909224996449542 with parameters: {'max_depth': 3, 'learning_rate': 0.013376621399495775, 'n_estimators': 105, 'subsample': 0.6204899998086485, 'alpha': 0.2699165905935789, 'random_state': 9758}.
[I 2019-05-24 04:54:08,209] Finished a trial resulted in value: 2.3507611407633955. C

[I 2019-05-24 06:12:50,268] Finished a trial resulted in value: 1.310210197066718. Current best value is 0.12909224996449542 with parameters: {'max_depth': 3, 'learning_rate': 0.013376621399495775, 'n_estimators': 105, 'subsample': 0.6204899998086485, 'alpha': 0.2699165905935789, 'random_state': 9758}.
[I 2019-05-24 06:15:32,032] Finished a trial resulted in value: 0.44211542091782835. Current best value is 0.12909224996449542 with parameters: {'max_depth': 3, 'learning_rate': 0.013376621399495775, 'n_estimators': 105, 'subsample': 0.6204899998086485, 'alpha': 0.2699165905935789, 'random_state': 9758}.
[I 2019-05-24 06:19:54,219] Finished a trial resulted in value: 0.462112973909386. Current best value is 0.12909224996449542 with parameters: {'max_depth': 3, 'learning_rate': 0.013376621399495775, 'n_estimators': 105, 'subsample': 0.6204899998086485, 'alpha': 0.2699165905935789, 'random_state': 9758}.
[I 2019-05-24 06:21:16,914] Finished a trial resulted in value: 0.6072980675945302. Cu

[I 2019-05-24 07:06:40,934] Finished a trial resulted in value: 0.5285316413503324. Current best value is 0.12909224996449542 with parameters: {'max_depth': 3, 'learning_rate': 0.013376621399495775, 'n_estimators': 105, 'subsample': 0.6204899998086485, 'alpha': 0.2699165905935789, 'random_state': 9758}.


In [33]:
for trial_i in mytrial:
    db.insert(trial_i)

In [34]:
df_trial = db.select()
df_trial[(df_trial['remark']=='tune 23')&(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'])[['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head(10)

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
181,2019-05-18 08:03:47.057550,tune 23,18,2.032553,0.002867,2.080601,0.011945,0.048049
149,2019-05-18 07:51:22.210733,tune 23,18,2.039637,0.002869,2.085890,0.011561,0.046253
167,2019-05-18 07:59:46.461391,tune 23,18,2.040172,0.002880,2.086240,0.011934,0.046068
43,2019-05-18 07:09:26.028333,tune 23,18,2.048812,0.002893,2.092745,0.011581,0.043933
203,2019-05-18 08:09:00.363434,tune 23,18,2.050189,0.002867,2.093182,0.011648,0.042993
49,2019-05-18 07:12:13.171108,tune 23,18,2.059042,0.002854,2.101067,0.011460,0.042025
108,2019-05-18 07:36:16.021443,tune 23,18,2.060967,0.002876,2.103015,0.011546,0.042049
99,2019-05-18 07:34:39.017661,tune 23,18,2.077046,0.002841,2.116267,0.011351,0.039221
98,2019-05-18 07:34:25.290910,tune 23,18,2.078141,0.002853,2.117434,0.011301,0.039293
195,2019-05-18 08:06:49.878070,tune 23,18,2.082113,0.002833,2.120605,0.011109,0.038493


In [35]:
db.commit()